In [ ]:
from google.colab import files 
uploaded = files.upload()


Saving train2000.csv to train2000.csv


In [ ]:
import pandas as pd 
import numpy as np
import io   
train= pd.read_csv(io.BytesIO(uploaded['train2000.csv'])) 
print(train)

      Unnamed: 0                     index  ... answer_start  c_id
0              0  56be85543aeaaa14008c9063  ...        269.0     0
1              1  56be88473aeaaa14008c9080  ...        104.0     2
2              2  56be892d3aeaaa14008c908c  ...        204.0     3
3              3  56bf74d53aeaaa14008c965a  ...        101.0     3
4              4  56bf76ef3aeaaa14008c9664  ...        148.0     4
...          ...                       ...  ...          ...   ...
1995        1995  56e1039be3433e1400422aa6  ...        103.0  2824
1996        1996  56e1039be3433e1400422aa7  ...        149.0  2824
1997        1997  56e104b7e3433e1400422acb  ...         43.0  2826
1998        1998  56e10f57cd28a01900c674ff  ...          3.0  2827
1999        1999  56e1156fe3433e1400422bb0  ...        112.0  2828

[2000 rows x 7 columns]


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
ques=train.question   # all questions
from nltk.tokenize import word_tokenize
tokenized_ques = [word_tokenize(i) for i in ques] #word tokenization of all questions


In [ ]:
from gensim.models import FastText
model= FastText(tokenized_ques, size=100, window=11, min_count=1,alpha=0.025, iter=100, workers=4,sg=1,cbow_mean=0,sorted_vocab=0) # training it with our model

In [ ]:
paragra= train.context.unique()  # all unique paragraphs
from nltk.tokenize import word_tokenize
tokenized_para = [word_tokenize(i) for i in paragra] #word tokenization of all paragraphs


In [ ]:
model.build_vocab(tokenized_para, update=True)  # Update the vocabulary
model.train(tokenized_para, total_examples=len(tokenized_para), epochs=model.epochs)

In [ ]:
# function to make sentence vectors from word vectors
def sent_vectorizer(sent, model):
    sent_vec =[]
    numw = 0
    for w in sent:
        try:
            if numw == 0:
                sent_vec = model[w]
            else:
                sent_vec = np.add(sent_vec, model[w])
            numw+=1
        except:
            pass
    
    return np.asarray(sent_vec) / numw

In [ ]:
paragraph=[]  # list of all unique paragraph vectors

for sentence in tokenized_para:
    paragraph.append(sent_vectorizer(sentence, model)) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [ ]:

question=[]  # list of all question vectors
for sentence in tokenized_ques:
    question.append(sent_vectorizer(sentence, model))   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
paraorigi=train.context # all paragraphs
tokenized_paraoriginal = [word_tokenize(i) for i in paraorigi]
print(len(tokenized_paraoriginal))

2000


In [ ]:
paraoriginal=[] # list of all paragraph vectors
for sentence in tokenized_paraoriginal:
    paraoriginal.append(sent_vectorizer(sentence, model))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
import math
def cosineValue(v1,v2):                  #compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

In [ ]:
from tqdm import tqdm

In [ ]:
# function  to find the accuracy of getting the correct paragraph out of the 3 and top 5 matched paragraphs 
count5=0   #  for top 5
count3=0   #  for top 3
for j in tqdm(range(len(question))):    #iterate over questions  # no. of questions=2000
    ques1=[]
    for i in range(len(paragraph)):     # iterate over paragraphs # no. of paragraph=1238
         ques1.append(cosineValue (question[j], paragraph[i]))  # storing cosine value for a particular question with each paragraph
    res = sorted(range(len(ques1)), key = lambda sub: ques1[sub])[-5:] #to store paragraph indices for top 5 cosine values
    arr0=paragraph[res[0]]  # corresponding 5 paragraph vectors
    arr1=paragraph[res[1]]
    arr2=paragraph[res[2]]
    arr3=paragraph[res[3]]
    arr4=paragraph[res[4]]
    
    para=paraoriginal[j]  # original paragraph vector (paragraph vector corresponding to the question)

    check0=para==arr0 #check whether the original vector is equal to one of the 5 vectors
    check1=para==arr1
    check2=para==arr2
    check3=para==arr3
    check4=para==arr4
    if(check2.all() or check3.all() or check4.all() ): # for top 3
        count3+=1
    if(check0.all() or check1.all() or check2.all() or check3.all() or check4.all() ): # for top 5
        count5+=1
    
print("\n")
print((count3/len(question))*100)
print((count5/len(question))*100)


100%|██████████| 2000/2000 [04:36<00:00,  7.24it/s]



73.3
79.4


In [ ]:
from nltk.corpus import stopwords 
stop_words= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [ ]:
# to make new vectors of questions by removing stopwords 
for i in range(len(ques)):
    ques[i]=ques[i].lower()         # convert to lower case
tokenized_ques = [word_tokenize(i) for i in ques]
ques_new=[]
for i in range(len(ques)):
    filtered_sentence = [w for w in tokenized_ques[i] if not w in stop_words]  # storing words which are not stopwords

    filtered_sentence = [] 

    for w in tokenized_ques[i]: 
        if w not in stop_words: 
            filtered_sentence.append(w)   
    ques_new.append(filtered_sentence)
question_new=[]                         #  list of all new question vectors
for sentence in ques_new:
    question_new.append(sent_vectorizer(sentence, model))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
 # same for all unique paragraph - removing stopwords
for i in range(len(paragra)):
    paragra[i]=paragra[i].lower()
tokenized_para = [word_tokenize(i) for i in paragra]
paragra_new=[]
for i in range(len(paragra)):
    filtered_sentence = [w for w in tokenized_para[i] if not w in stop_words] 

    filtered_sentence = [] 

    for w in tokenized_para[i]: 
        if w not in stop_words: 
            filtered_sentence.append(w)   
    paragra_new.append(filtered_sentence)
paragraph_new=[]                            #  list of all new unique paragraph vectors
for sentence in paragra_new:
    paragraph_new.append(sent_vectorizer(sentence, model))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
#same for all paragraphs - removing stopwords
for i in range(len(paraorigi)):
    paraorigi[i]=paraorigi[i].lower()
tokenized_paraoriginal = [word_tokenize(i) for i in paraorigi]
paragraorigi_new=[]
for i in range(len(paraoriginal)):
    filtered_sentence = [w for w in tokenized_paraoriginal[i] if not w in stop_words] 

    filtered_sentence = [] 

    for w in tokenized_paraoriginal[i]: 
        if w not in stop_words: 
            filtered_sentence.append(w)   
    paragraorigi_new.append(filtered_sentence)
paraoriginal_new=[]                               #  list of all new paragraph vectors
for sentence in paragraorigi_new:
    paraoriginal_new.append(sent_vectorizer(sentence, model))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
# same function  to find the accuracy but with questions and paragraphs after removing stopwords
count3=0
count5=0
for j in tqdm(range(len(question_new))):
    ques1=[]
    for i in range(len(paragraph_new)):
         ques1.append(cosineValue (question_new[j], paragraph_new[i]))
    res = sorted(range(len(ques1)), key = lambda sub: ques1[sub])[-5:] 
    arr0=paragraph_new[res[0]]
    arr1=paragraph_new[res[1]]
    arr2=paragraph_new[res[2]]
    arr3=paragraph_new[res[3]]
    arr4=paragraph_new[res[4]]
    para=paraoriginal_new[j]

    check0=para==arr0
    check1=para==arr1
    check2=para==arr2
    check3=para==arr3
    check4=para==arr4

    if(check2.all() or check3.all() or check4.all() ): # for top 3
        count3+=1
    if(check0.all() or check1.all() or check2.all() or check3.all() or check4.all() ): # for top 5
        count5+=1
    
print("\n")
print((count3/len(question))*100)
print((count5/len(question))*100)

100%|██████████| 2000/2000 [04:37<00:00,  7.21it/s]



79.2
84.55


In [ ]:
 # for each paragraph, breaking the paragraph into sentences and storing the vector of each sentence   (after removing stopwords)
paravectors=[] # contains list of parasentence_new[i] which is containing list of sentence vectors of the particular paragraph
for i in tqdm(range(len(paragra))):
    text=paragra[i]
    a_list1 = nltk.tokenize.sent_tokenize(text)  # breaking the paragraph to sentences
    tokenized_sents3 = [word_tokenize(j) for j in a_list1]
    parasent=[]            
    for i in range(len(tokenized_sents3)):
        filtered_sentence = [w for w in tokenized_sents3[i] if not w in stop_words] 
        filtered_sentence = [] 
        for w in tokenized_sents3[i]: 
            if w not in stop_words: 
                filtered_sentence.append(w)   
        parasent.append(filtered_sentence)    # storing each sentence after removing stopwords
    parasentence_new=[]                    
    for sentence in parasent:
        parasentence_new.append(sent_vectorizer(sentence, model)) # contains list of sentence vectors of the particular paragraph
    paravectors.append(parasentence_new)

  0%|          | 0/1238 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
100%|██████████| 1238/1238 [00:01<00:00, 710.47it/s]


In [ ]:
 # function which finds the accuracy of the most similar sentence in a given paragraph with a given question
def cosineValue1(v1,parasentence_new):
    cossimilar=[]
    for i in range(len(parasentence_new)):
        cossimilar.append(cosineValue(v1,parasentence_new[i]))
    res1 = sorted(range(len(cossimilar)), key = lambda sub: cossimilar[sub])[-1:]
    x= cossimilar[res1[0]]
    return x

In [ ]:
# function  to find the accuracy of getting the correct paragraph (after considering only the most similar sentence in it) out of the 3 and top 5 matched paragraphs (and removing stopwords)
count3=0
count5=0
for j in tqdm(range(len(question_new))):
    ques1=[]
    for i in range(len(paragraph_new)):
         ques1.append(cosineValue1 (question_new[j],paravectors[i]))  # the parameters are the given question vector and list containing sentence vectors of a given paragraph
    res = sorted(range(len(ques1)), key = lambda sub: ques1[sub])[-5:] 
    arr0=paragraph_new[res[0]]
    arr1=paragraph_new[res[1]]
    arr2=paragraph_new[res[2]]
    arr3=paragraph_new[res[3]]
    arr4=paragraph_new[res[4]]
    para_j=paraoriginal_new[j]

    check0=para_j==arr0
    check1=para_j==arr1
    check2=para_j==arr2
    check3=para_j==arr3
    check4=para_j==arr4
    
    if(check2.all() or check3.all() or check4.all() ): # for top 3
        count3+=1
    if(check0.all() or check1.all() or check2.all() or check3.all() or check4.all() ): # for top 5
        count5+=1
    
print("\n")
print((count3/len(question))*100)
print((count5/len(question))*100)

100%|██████████| 2000/2000 [17:31<00:00,  1.90it/s]



81.0
86.9
